In [72]:
from string import punctuation
from gensim.models.doc2vec import TaggedDocument
import json
import gensim
import nltk,string
from random import shuffle
from gensim.models import doc2vec
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, \
Dropout, Activation, Input, Flatten, Concatenate
from keras.regularizers import l2
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd


########### pretrain doc2vector ############

with open("test.json", 'r') as f:
    reviews=[]
    for line in f: 
        review = json.loads(line) 
        try:
            review["text"].strip().lower().encode('ascII')
        except:
            ;
        else:
            reviews.append(review["text"])
    
    
sentences=[ [token.strip(string.punctuation).strip() \
             for token in nltk.word_tokenize(doc.lower()) \
                 if token not in string.punctuation and \
                 len(token.strip(string.punctuation).strip())>=2]\
             for doc in reviews]
            
docs=[TaggedDocument(sentences[i], [str(i)]) for i in range(len(sentences)) ]
wv_model = doc2vec.Doc2Vec(dm=1, min_count=5, window=5, size=200, workers=4)
wv_model.build_vocab(docs) 


for epoch in range(30):
    shuffle(docs)
    wv_model.train(docs, total_examples=len(docs), epochs=1)
        
print("Top 5 words similar to word 'price'")
print wv_model.wv.most_similar('price', topn=5)

print("Top 5 words similar to word 'price' but not relevant to 'bathroom'")
print wv_model.wv.most_similar(positive=['price','money'], negative=['bathroom'], topn=5)

print("Similarity between 'price' and 'bathroom':")
print wv_model.wv.similarity('price','bathroom') 

print("Similarity between 'price' and 'charge':")
print wv_model.wv.similarity('price','charge') 

print wv_model.wv

############## CNN #################



BEST_MODEL_FILEPATH="best_model"
MAX_NB_WORDS=8000
MAX_DOC_LEN=1000

EMBEDDING_DIM=200
MAX_NB_WORDS=8000

EMBEDDING_DIM=100
FILTER_SIZES=[2,3,4]

BTACH_SIZE = 64
NUM_EPOCHES = 20


data=pd.read_csv("data_to_be_labeled.csv",header=None)
data.head()

labels = []

for subdata in data[2][0:2]:
    label = []
    for dd in subdata.split(";"):
        for d in dd.split(","):
            label.append(d.strip())
    labels.append(label)
    
print labels   

mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(labels)
Y.shape
mlb.classes_
np.sum(Y, axis=0)


tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(data[1][0:2])
NUM_WORDS = min(MAX_NB_WORDS, len(tokenizer.word_index))
embedding_matrix = np.zeros((NUM_WORDS+1, EMBEDDING_DIM))

for word, i in tokenizer.word_index.items():
    if i >= NUM_WORDS:
        continue
    if word in wv_model.wv:
        embedding_matrix[i]=wv_model.wv[word]


voc=tokenizer.word_index
sequences = tokenizer.texts_to_sequences(data[1][0:2])
padded_sequences = pad_sequences(sequences, \
                                 maxlen=MAX_DOC_LEN, \
                                 padding='post', truncating='post')

NUM_OUTPUT_UNITS=len(mlb.classes_)



X_train, X_test, Y_train, Y_test = train_test_split(\
                padded_sequences, Y, test_size=0.3, random_state=0)

model=cnn_model(FILTER_SIZES, MAX_NB_WORDS, \
                MAX_DOC_LEN, NUM_OUTPUT_UNITS)

earlyStopping=EarlyStopping(monitor='val_loss', patience=0, verbose=2, mode='min')
checkpoint = ModelCheckpoint(BEST_MODEL_FILEPATH, monitor='val_acc', \
                             verbose=2, save_best_only=True, mode='max')
    
training=model.fit(X_train, Y_train, \
          batch_size=BTACH_SIZE, epochs=NUM_EPOCHES, \
          callbacks=[earlyStopping, checkpoint],\
          validation_data=[X_test, Y_test], verbose=2)

model.load_weights("best_model")

pred=model.predict(X_test)
pred[0:5]

from sklearn.metrics import classification_report

Y_pred=np.copy(pred)
Y_pred=np.where(Y_pred>0.5,1,0)

Y_pred[0:10]
Y_test[0:10]

print(classification_report(Y_test, Y_pred, \
                            target_names=mlb.classes_))



############### CNN function ##################

def cnn_model(FILTER_SIZES, \
              # filter sizes as a list
              MAX_NB_WORDS, \
              # total number of words
              MAX_DOC_LEN, \
              # max words in a doc
              NUM_OUTPUT_UNITS=1, \
              # number of output units
              EMBEDDING_DIM=200, \
              # word vector dimension
              NUM_FILTERS=64, \
              # number of filters for all size
              DROP_OUT=0.5, \
              # dropout rate
              PRETRAINED_WORD_VECTOR=None,\
              # Whether to use pretrained word vectors
              LAM=0.01):            
              # regularization coefficient
    
    main_input = Input(shape=(MAX_DOC_LEN,), \
                       dtype='int32', name='main_input')
    
    if PRETRAINED_WORD_VECTOR is not None:
        embed_1 = Embedding(input_dim=MAX_NB_WORDS+1, \
                        output_dim=EMBEDDING_DIM, \
                        input_length=MAX_DOC_LEN, \
                        weights=[PRETRAINED_WORD_VECTOR],\
                        trainable=False,\
                        name='embedding')(main_input)
    else:
        embed_1 = Embedding(input_dim=MAX_NB_WORDS+1, \
                        output_dim=EMBEDDING_DIM, \
                        input_length=MAX_DOC_LEN, \
                        name='embedding')(main_input)
    # add convolution-pooling-flat block
    conv_blocks = []
    for f in FILTER_SIZES:
        conv = Conv1D(filters=NUM_FILTERS, kernel_size=f, \
                      activation='relu', name='conv_'+str(f))(embed_1)
        conv = MaxPooling1D(MAX_DOC_LEN-f+1, name='max_'+str(f))(conv)
        conv = Flatten(name='flat_'+str(f))(conv)
        conv_blocks.append(conv)

    z=Concatenate(name='concate')(conv_blocks)
    drop=Dropout(rate=DROP_OUT, name='dropout')(z)

    dense = Dense(192, activation='relu',\
                    kernel_regularizer=l2(LAM),name='dense')(drop)
    preds = Dense(NUM_OUTPUT_UNITS, activation='sigmoid', name='output')(dense)
    model = Model(inputs=main_input, outputs=preds)
    
    model.compile(loss="binary_crossentropy", \
              optimizer="adam", metrics=["accuracy"]) 
    
    return model

Top 5 words similar to word 'price'
[(u'service', 0.7669029831886292), (u'value', 0.6675570011138916), (u'quality', 0.6538277268409729), (u'food', 0.6511838436126709), (u'reasonable', 0.6412127614021301)]
Top 5 words similar to word 'price' but not relevant to 'bathroom'
[(u'paying', 0.6908645629882812), (u'request', 0.6339963674545288), (u'sit', 0.6279985904693604), (u'mind', 0.6259720921516418), (u'yes', 0.6228559017181396)]
Similarity between 'price' and 'bathroom':
0.523103921302
Similarity between 'price' and 'charge':
0.466787467466
[['', 'location', 'price', 'location', 'price', 'staff', 'amenities', 'amentities', 'service', '', ''], ['', 'location', 'location', 'enviornment', 'amentities', 'service', 'amentities', '']]
Train on 1 samples, validate on 1 samples
Epoch 1/20
Epoch 00001: val_acc improved from -inf to 0.25000, saving model to best_model
 - 1s - loss: 2.6595 - acc: 0.1250 - val_loss: 2.6048 - val_acc: 0.2500
Epoch 2/20
Epoch 00002: val_acc did not improve
 - 0s - los